In [1]:
import os

import nglview as nv
import numpy
import openmm
import openmm.app as app
import openmm.unit as unit
import parmed as pmd
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
from paprika.build import align
from paprika.evaluator import Setup
from pkg_resources import resource_filename
from rdkit import Chem

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.
/home/jsetiadi/mambaforge/envs/paprika-dev/lib/python3.10/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jsetiadi/mambaforge/envs/paprika-dev/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/jsetiadi/mambaforge/envs/paprika-dev/lib/python3.10/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the sma

In [2]:
build_folder = "build_files"
simulation_folder = "simulation"
host_resname = "MGO"
dummy_name = "DUM"
os.makedirs(simulation_folder, exist_ok=True)

### Add Dummy Atom to PDB file

In [3]:
structure = pmd.load_file(f"{build_folder}/bcd.am1bcc.sybyl.mol2", structure=True)

In [4]:
# Align bCD to origin
structure = align.translate_to_origin(structure, weight="geo", atom_mask="@/C")

# Add dummy atom to structure
Setup.add_dummy_atoms_to_structure(
    structure,
    [
        numpy.array([0, 0, 0]),
    ],
)

In [5]:
nv.show_parmed(structure)

NGLWidget()

In [6]:
# Save PDBFile
with open(f"{simulation_folder}/system.pdb", "w") as f:
    app.PDBFile.writeFile(structure.topology, structure.positions, f, keepIds=True)

## Create OpenMM System with Sage-OBC2

In [7]:
molecule = Molecule.from_file(f"{build_folder}/bcd.am1bcc.sybyl.mol2")

GBSA = resource_filename(
    "openff.toolkit", os.path.join("data", "test_forcefields", "GBSA_OBC2-1.0.offxml")
)
forcefield = ForceField("openff-2.0.0.offxml", GBSA)

# Don't use ACE model at all
gbsa_handler = forcefield.get_parameter_handler("GBSA")
gbsa_handler.sa_model = None

openmm_system = forcefield.create_openmm_system(
    molecule.to_topology(), charge_from_molecules=[molecule]
)

### Add Dummy Atom To `System`

In [8]:
# Add Mass of Lead to System Particle
openmm_system.addParticle(207.2 * unit.dalton)

147

### Add Dummy Particle to `NonbondedForce` and `GBSAOBCForce`

In [9]:
# Add Dummy particle to NonbondedForce and GBSAOBCForce
for force in openmm_system.getForces():
    if isinstance(force, openmm.NonbondedForce):
        # Add particle with no charges or LJ energy
        force.addParticle(
            0.0 * unit.elementary_charge,
            2.0 * unit.angstrom,
            0.0 * unit.kilocalorie_per_mole,
        )
    elif isinstance(force, openmm.GBSAOBCForce):
        # Add GBSA particle with no charge and oxygen parameters
        force.addParticle(0.0 * unit.elementary_charge, 1.5 * unit.angstrom, 0.85)

### Convert Dummy Particle to a Hard Sphere

In [10]:
host_atoms = [
    atom.index
    for atom in structure.topology.atoms()
    if atom.residue.name == host_resname
]
dummy_atom = [
    atom.index for atom in structure.topology.atoms() if atom.name == dummy_name
]

sigma_wall = 3.0 * unit.angstrom
epsilon_wall = 1.0 * unit.kilocalorie_per_mole

In [11]:
hard_sphere = openmm.CustomNonbondedForce(
    "lambda_sterics * U_repulsive;"
    "U_repulsive = step(R_particle - r) * (U_Mie + epsilon_wall);"
    "U_Mie = prefactor * epsilon_wall * (1/repulsive - 1/dispersive);"
    "repulsive = (dispersive)^(coeff_r/coeff_a);"
    "dispersive = softcore_alpha*(1.0-lambda_sterics) + (r_prime/sigma_wall)^coeff_a;"
    "prefactor = (coeff_r/(coeff_r - coeff_a)) * (coeff_r/coeff_a)^(coeff_a/(coeff_r-coeff_a));"
    "r_prime = r - (R_particle - R_min);"
    "R_min = sigma_wall * ((coeff_r/coeff_a)^(coeff_a/(coeff_r-coeff_a)) - softcore_alpha*(1.0-lambda_sterics))^(1/coeff_a);"
)
hard_sphere.addGlobalParameter("lambda_sterics", 1.0)
hard_sphere.addGlobalParameter("softcore_alpha", 0.5)
hard_sphere.addGlobalParameter("R_particle", 1.0 * unit.angstrom)
hard_sphere.addGlobalParameter("coeff_r", 12)
hard_sphere.addGlobalParameter("coeff_a", 6)
hard_sphere.addGlobalParameter("sigma_wall", sigma_wall)
hard_sphere.addGlobalParameter("epsilon_wall", epsilon_wall)
hard_sphere.addPerParticleParameter("sigma")
hard_sphere.addPerParticleParameter("epsilon")
hard_sphere.setNonbondedMethod(openmm.CustomNonbondedForce.NoCutoff)
hard_sphere.setUseLongRangeCorrection(False)
hard_sphere.setForceGroup(10)
hard_sphere.addInteractionGroup(dummy_atom, host_atoms)

# Set LJ parameters
nonbonded = [
    force
    for force in openmm_system.getForces()
    if isinstance(force, openmm.NonbondedForce)
][0]
for atom in range(nonbonded.getNumParticles()):
    charge, sigma, epsilon = nonbonded.getParticleParameters(atom)
    if atom in dummy_atom:
        hard_sphere.addParticle([sigma_wall, epsilon_wall])
    else:
        hard_sphere.addParticle([sigma, epsilon])

openmm_system.addForce(hard_sphere)

5

### Add Center of Mass restraints

In [12]:
mol = Chem.MolFromMol2File(f"{build_folder}/bcd.am1bcc.sybyl.mol2", removeHs=False)

In [13]:
central_oxygen_mol = Chem.MolFromSmarts("[#8:1]([#6]1-[#8]-[#6]-[#6]-[#6]-[#6]-1)")
central_match = mol.GetSubstructMatches(central_oxygen_mol)
central_oxygen = [list(ai)[0] for ai in central_match]

In [14]:
com_restraint = openmm.CustomCentroidBondForce(2, "0.5*k_com*distance(g1,g2)^2;")
com_restraint.addPerBondParameter("k_com")
com_restraint.setUsesPeriodicBoundaryConditions(False)

# Add host and dummy indices
g1 = com_restraint.addGroup(central_oxygen, [1 for i in range(len(central_oxygen))])
g2 = com_restraint.addGroup(dummy_atom)

com_restraint.addBond([g1, g2], [100.0 * unit.kilocalorie_per_mole / unit.angstrom**2])
com_restraint.setForceGroup(11)

openmm_system.addForce(com_restraint)

6

In [15]:
# Save system
with open(f"{simulation_folder}/system.xml", "w") as f:
    f.write(openmm.XmlSerializer.serialize(openmm_system))